## Unit Tests

test each backend component

In [1]:
import json

with open("config.json") as f:
    config = json.load(f)

#### Vector DB

Must keep run_vecdb.py running

In [1]:
from backend.vecdb.vecdb import VecDB

# Create a VecDB instance
vecdb_instance = VecDB(config)

# try insert some toy data and then query
for i in range(5):
    print("-"*40, "\n", f"Round {i+1}:", sep="")

    # insert
    vecdb_instance.insert(ids=[3*i, 3*i+1, 3*i+2],
                          embeddings=[[0.1*i]*1536 for _ in range(3)],
                          months=[202310, 202311, 202311],
                          company_ids=[0, 0, 1],
                          datatypes=[0, 0, 1])
    
    # query
    query_filters = ['', 'company==0', 'company==1 and month==202311 and datatype==1']
    for query_filter in query_filters:
        print(f"Query filter: {query_filter}")
        topk_results = vecdb_instance.query(query_vector=[1]*1536,
                                            query_filter=query_filter)
        for k, result in enumerate(topk_results):
            print(f"- Rank {k+1}: {result}")

# delete all data and stop server
vecdb_instance.remove_all_data(rebuild_collection=False)
vecdb_instance.disconnect()

Starting Vector DB service...
- Connected to: [('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x000001B1E15F60A0>)]
- Current collections: ['all_data']
- Loaded collection, num of vector: 7
- Done
----------------------------------------
Round 1:
- inserted 3 vectors, current num of vectors in db: 10
Query filter: 
- Rank 1: {'id': 2, 'distance': 1536.0, 'entity': {'datatype': 1, 'id': 2, 'month': 202311, 'company': 1}}
- Rank 2: {'id': 1, 'distance': 1536.0, 'entity': {'datatype': 0, 'id': 1, 'month': 202311, 'company': 0}}
- Rank 3: {'id': 0, 'distance': 1536.0, 'entity': {'datatype': 0, 'id': 0, 'month': 202310, 'company': 0}}
- Rank 4: {'id': 6, 'distance': 1538.964111328125, 'entity': {'datatype': 1, 'id': 6, 'month': 202306, 'company': 0}}
- Rank 5: {'id': 4, 'distance': 1539.099853515625, 'entity': {'datatype': 1, 'id': 4, 'month': 202306, 'company': 0}}
- Rank 6: {'id': 3, 'distance': 1539.20751953125, 'entity': {'datatype': 1, 'id': 3, 'month': 202306, 'compan

#### Data Update

Must keep server.py and run_vecdb.py running

In [1]:
from data_update.data_update import DataUpdate

data_update_instance = DataUpdate(config)
data_update_instance.update_test_data()

Starting Text DB service ...
- Done
Starting Vector DB service...
- Connected to: [('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x000001C86598E520>)]
- Current collections: ['all_data']
- Loaded collection, num of vector: 0
- Done
Processing document: sA_3978866
- insert document success, document id: 0
- insert embeddings into text db success, number of embeddings: 3
- inserted 3 vectors, current num of vectors in db: 3
Processing document: sA_3978529
- insert document success, document id: 1
- insert embeddings into text db success, number of embeddings: 6
- inserted 3 vectors, current num of vectors in db: 6
Processing document: sA_3979690
- insert document success, document id: 2
- insert embeddings into text db success, number of embeddings: 7
- inserted 1 vectors, current num of vectors in db: 7


remove all data if you want

In [2]:
data_update_instance.remove_all_textdb_data()
data_update_instance.remove_all_vecdb_data()

Removing all data from text db
- Done
Removing all data from vector db
- Current collections: ['all_data']
- Created index: [<pymilvus.orm.index.Index object at 0x00000228C9846D30>]
- Loaded collection, num of vector: 0
- Done


#### Respondent

init

In [2]:
from backend.vecdb.vecdb import VecDB
from backend.textdb.textdb import TextDB
from backend.textdb.text_preprocess import TextPreprocesser
from backend.respondent.respondent import Respondent

with open('backend/openai.key') as f:
    openai_key = f.read().strip()

# Create instances
vecdb_instance = VecDB(config)
textdb_instance = TextDB(config)
text_preprocess_instance = TextPreprocesser(config, openai_key)
respondent_instance = Respondent(config, vecdb_instance, textdb_instance, text_preprocess_instance, openai_key)

Starting Vector DB service...
- Connected to: [('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x7f38c31fb490>)]
- Current collections: ['all_data', 'all_data_with_copyright']
- Loaded collection, num of vector: 3670000
- Done
Starting Text DB service ...
- Done


product mode

In [3]:
user_query_text = '药明生物上周五为何大跌?'
model_names = ['gpt-4-1106-preview']
result = respondent_instance.query(user_query_text, model_names, 
                                   answer_language='auto', copyright_names=['public'],
                                   company_ids=[], datatype_ids=[], months=[],
                                   chat_id=0, auto_filter=True, product_mode=True)

print("* Status:", result["Status"])
print("* Message:", result["Message"])
# if "references" in result:
#     print("* References:")
#     for i, each in enumerate(result["references"]):
#         print(f"[{i+1}]:", each['title'])
#         print(each['url'])
#         print("-"*40)
if "summaries" in result:
    print("* Summaries:")
    for i, each in enumerate(result["summaries"]):
        print("["+each['model']+"]:")
        print(each['summary'])
        print("-"*40)
print(json.dumps(result, indent=4))

* Status: Success
* Message: 
* Summaries:
[gpt-4-1106-preview]:
根据文档 [8]，生物医药港股在1月19日午后的跌幅扩大，其中药明生物（02269.HK）下跌了3%。尽管文档中没有提供药明生物下跌的具体原因，但通常股票价格的波动可能受到多种因素的影响，包括市场整体情绪、行业动态、公司业绩预期、宏观经济状况等。

从提供的信息来看，药明生物在近期有积极的业绩指引和项目增长（参考：[3]、[4]、[5]、[6]、[7]），并且在分析师的报告中获得了乐观的评价和目标价格上调。然而，文档 [1] 提到，整个医药行业的市场风险偏好处于较低水平，且由于多种因素的交织影响，业绩预期面临不确定因素较多。这可能意味着尽管药明生物的业绩和前景被看好，但整体市场对医药板块的信心不足，可能导致了股价的波动。

此外，文档 [8] 中提到其他生物医药公司的股价也出现了下跌，这可能表明市场对整个生物医药板块的情绪在短期内转为谨慎，从而影响了药明生物的股价。

综上所述，药明生物上周五的股价下跌可能与市场对医药行业整体的风险偏好降低有关，而非公司单独的负面消息所致（参考：[1]、[8]）。
----------------------------------------
{
    "Status": "Success",
    "Message": "",
    "references": [
        {
            "chunk_ids": [
                4903600
            ],
            "texts": "\u4ece\u5f71\u54cd\u56e0\u5b50\u770b\uff0c\u4f30\u503c\u7684\u4e0b\u8dcc\u5e45\u5ea6\u663e\u8457\u9ad8\u4e8e\u4e1a\u7ee9\u7684\u4e0b\u6ed1\u5e45\u5ea6\uff0c\u5373\u4fbf\u662f\u4e1a\u7ee9\u6709\u6b63\u589e\u957f\u7684\u7ec6\u5206\u9886\u57df\u548c\u516c\u53f8\uff0c\u56e0\u4f

full mode

In [6]:
user_query_text = '药明生物上周五为何大跌?'
model_names = ['gpt-4-1106-preview']
result = respondent_instance.query(user_query_text, model_names, 
                                   answer_language='auto', copyright_names=['public'],
                                   company_ids=[], datatype_ids=[], months=[],
                                   chat_id=0, auto_filter=True, product_mode=False)

print("* Status:", result["Status"])
print("* Message:", result["Message"])
# if "references" in result:
#     print("* References:")
#     for i, each in enumerate(result["references"]):
#         print(f"[{i+1}]:", each['title'])
#         print(each['texts'])
#         print(each['url'])
#         print("Date:", each['date'])
#         print("-"*40)
if "summaries" in result:
    print("* Summaries:")
    for i, each in enumerate(result["summaries"]):
        print("["+each['model']+"]:")
        print(each['summary'])
        print("Time:", each['time'])
        print("-"*40)
print(json.dumps(result, indent=4))

- current chat history num: 1
- After transformation: Why did WuXi Biologics' stock price drop significantly last Friday?
- query filter:  (copyright == 9)  and  (company == 479)  and  (month == 202401) 
* Status: Success
* Message: 
* Summaries:
[gpt-4-1106-preview]:
根据文档[8]，药明生物在上周五即2024年1月26日午后出现了大幅下跌，跌幅超过了23%。虽然该文档没有提供具体的下跌原因，但是这种大幅度的下跌通常与市场对公司的负面看法或者公司内部出现的重大不利消息有关。考虑到药明生物是一家生物医药公司，其股价可能会受到行业动态、公司业绩、管理层变动、监管政策变化、市场竞争状况等多种因素的影响。

需要注意的是，文档[6]和[7]提到在1月19日和1月22日，生物医药港股以及医药外包概念股出现了跌幅，其中药明生物也有所下跌。这可能表明在上周五之前，市场对于生物医药板块已经有了一定程度的负面情绪。

此外，文档[1]、[2]、[3]、[4]和[5]在之前对药明生物的业绩和前景给出了积极的评价，但这些评价并不能完全防止股价在短期内受到市场情绪和其他外部因素的影响。

综上所述，虽然没有直接的信息说明药明生物上周五大跌的具体原因，但可以推测可能是由于市场对整个生物医药板块的负面情绪影响，以及可能存在的其他未公开的不利因素所导致的。(Reference: [6], [7], [8])。
Time: 26.066750288009644
----------------------------------------
{
    "Status": "Success",
    "Message": "",
    "transform_time": 6.812314987182617,
    "transformed_query_text": "Why did WuXi Biologics' stock price drop significantly last Friday?",
    "query_f

given document ids

In [7]:
user_query_text = '半导体公司如何？'
doc_str_ids = ['bt_175993', 'jf_383599']
model_names = ['gpt-4-1106-preview']
copyright_names = ['bernste', 'jefferies', 'public']
top_k = -1
result = respondent_instance.query_with_doc_ids(user_query_text, model_names, doc_str_ids, 
                                                answer_language='auto', copyright_names=copyright_names,
                                                chat_id=100, top_k=top_k)

print("* Status:", result["Status"])
print("* Message:", result["Message"])
# if "references" in result:
#     print("* References:")
#     for i, each in enumerate(result["references"]):
#         print(f"[{i+1}]:", each['title'])
#         print(each['texts'])
#         print(each['url'])
#         print("Date:", each['date'])
#         print("-"*40)
if "summaries" in result:
    print("* Summaries:")
    for i, each in enumerate(result["summaries"]):
        print("["+each['model']+"]:")
        print(each['summary'])
        print("-"*40)
print(json.dumps(result, indent=4))

* Status: Success
* Message: 
* Summaries:
[gpt-4-1106-preview]:
根据最新的报告，半导体公司在2023年第一季度（1Q23）的整体表现是积极的。收入中位数超出了市场预期的2.0%，而第二季度（2Q23）的收入预期中位数也高于共识预期1.5%（参考：[1]）。这表明半导体公司的业绩超出了市场预期。

在盈利能力方面，半导体公司的毛利率和运营利润率预计在2023年第二季度（2Q23）达到底部，并预计将在2023年恢复到2022年的水平（参考：[2]）。这意味着尽管短期内可能面临压力，但整体盈利能力预计将得到改善。

库存管理方面，除了NVDA、MRVL、AMAT、FORM、CAMT、LAZR和NVTS之外，所有报告了2023年第一季度（1Q23）业绩的半导体公司都出现了季度环比和年度同比的库存天数（DOI）增加。这可能最终导致交货时间的缩短，并最终导致库存校正（参考：[3]）。

股票表现方面，自2022年10月18日SOX相对于SPX的相对低点以来，NVDA、RMBS、NVTS、SGH、ALGM和AMD等公司的表现领先于覆盖组内的其他公司，分别实现了255%、120%、111%、98%、95%和91%的增长（参考：[4]）。

政府对建设本地市场半导体制造设施提供的激励措施也在增加，预计未来5年将有超过3000亿美元（300 billion dollars）的总激励措施，相当于每年660亿美元（66 billion dollars）（参考：[6]）。

总的来说，半导体公司在收入和盈利能力方面表现良好，库存管理也在改善，同时受益于政府的激励政策，展望未来有积极的增长前景。
----------------------------------------
{
    "Status": "Success",
    "Message": "",
    "references": [
        {
            "chunk_ids": [
                8979087
            ],
            "texts": "appreciates secular growth from: 1) \u201cMoore's Bend\u201d/bigge